## Toronto Crime Analysis

### This notebook outputs a csv that relates Toronto neighborhoods to census tracts and their corresponding Canada Census ID, to be used by other notebooks that pull census data

In [2]:
import requests
import json
import pandas as pd
from pprint import pprint

In [3]:
# import crimes csv
crimes_df = pd.DataFrame(pd.read_csv('../output/crimes_csv.csv', index_col=0))
crimes_df.head()

,Hood_ID,Index_,MCI,Neighbourhood,event_unique_id,occurrencedate,occurrenceday,occurrencedayofweek,occurrencedayofyear,occurrencehour,occurrencemonth,occurrenceyear,offence,premisetype
0,84,976,Assault,Little Portugal (84),GO-2015942420,1388552400000,1,Wednesday,1,0,January,2014,Assault,Apartment
1,25,977,Assault,Glenfield-Jane Heights (25),GO-20162295518,1388552400000,1,Wednesday,1,0,January,2014,Assault,Apartment
2,71,978,Assault,Cabbagetown-South St.James Town (71),GO-20142458487,1388552400000,1,Wednesday,1,0,January,2014,Assault,Apartment
3,71,979,Assault,Cabbagetown-South St.James Town (71),GO-20142458487,1388552400000,1,Wednesday,1,0,January,2014,Assault,Apartment
4,58,980,Theft Over,Old East York (58),GO-20141573896,1388552400000,1,Wednesday,1,0,January,2014,Theft Over,House


In [4]:
crimes_df['Hood_ID'].nunique()
# crimes data has 140 neighborhoods

140

In [5]:
len(crimes_df)

130167

#### Crime data has 130167 incidents, each incident has a neighborhood ID
#### Data from Canada Census API must be fetched per geographical area, the smallest area you can pull is a census tract. 
#### Before we start fetching census data, let's get a list of the census tracts that we require

#### First, we need a way to convert between neighborhood ID and census tract. Found a CSV file with the conversion.

In [7]:
conversion = pd.read_csv('../additional-datasets/TorontoCensusTractsConversionTable.csv') 

In [8]:
conversion_df_raw = pd.DataFrame(conversion)
conversion_df_raw.head()

,"CT UID - Census Tract Unique Identifier (""CMA Code""+""0""+""Census Tract ID""), n=531","CT ID - Census Tract Identifier, n=531","Neighb ID, n=140","Neighbourhood Name, n=140","CPA ID, n=46","CPA Name (Community Planning Area), n=46","Sub-LHIN ID, n=14","Sub-LHIN Name, n=14","LHIN ID, n=5","LHIN Name, n=5"
0,5350001.0,1.0,70,South Riverdale,752,South Riverdale Greenwood,705,South East,7,Toronto Central
1,5350002.0,2.0,77,Waterfront Communities-The Island,733,Downtown Centre,703,South West,7,Toronto Central
2,5350003.0,3.0,85,South Parkdale,731,Parkdale Roncesvalles,703,South West,7,Toronto Central
3,5350004.0,4.0,85,South Parkdale,731,Parkdale Roncesvalles,703,South West,7,Toronto Central
4,5350005.0,5.0,85,South Parkdale,731,Parkdale Roncesvalles,703,South West,7,Toronto Central


In [9]:
conversion_df_raw.columns

Index([' CT UID - Census Tract Unique Identifier\n("CMA Code"+"0"+"Census Tract ID"),\nn=531',
       'CT ID - Census Tract Identifier,\nn=531', 'Neighb ID, \n n=140',
       'Neighbourhood Name,\n n=140', 'CPA ID,\nn=46',
       'CPA Name \n(Community Planning Area),\n n=46', 'Sub-LHIN ID,\nn=14',
       'Sub-LHIN Name, \nn=14', 'LHIN ID,\nn=5', 'LHIN Name, \nn=5'],
      dtype='object')

In [10]:
# rename the columns we need
renamed_conversion_df = conversion_df_raw.rename(columns={' CT UID - Census Tract Unique Identifier\n("CMA Code"+"0"+"Census Tract ID"),\nn=531':'census_tract',
                             'Neighb ID, \n n=140':'hood_id',
                             'Neighbourhood Name,\n n=140': 'neighbourhood'})

In [11]:
# extract the columns we need
conversion_df = renamed_conversion_df[['census_tract', 'hood_id', 'neighbourhood']]
conversion_df

,census_tract,hood_id,neighbourhood
0,5350001.00,70,South Riverdale
1,5350002.00,77,Waterfront Communities-The Island
2,5350003.00,85,South Parkdale
3,5350004.00,85,South Parkdale
4,5350005.00,85,South Parkdale
5,5350006.00,85,South Parkdale
6,5350007.01,85,South Parkdale
7,5350007.02,85,South Parkdale
8,5350008.00,82,Niagara
9,5350009.00,82,Niagara


In [12]:
conversion_df['hood_id'].nunique()
# converstion df also has 140 neighborhoods, yay!

140

In [13]:
conversion_df['census_tract'].nunique()
# they are divided into 531 census tracts

531

In [14]:
# lets do a quick test to make sure that the neighborhood id's are equivalent to the MCI id's
# we can test name of neighborhood for a given neighborhood id from each dataframe
conversion_df['neighbourhood'][conversion_df['hood_id']==44]

288    Flemingdon Park
289    Flemingdon Park
290    Flemingdon Park
291    Flemingdon Park
Name: neighbourhood, dtype: object

In [15]:
crimes_df['Neighbourhood'][crimes_df['Hood_ID']==44]

39        Flemingdon Park (44)
40        Flemingdon Park (44)
53        Flemingdon Park (44)
110       Flemingdon Park (44)
231       Flemingdon Park (44)
232       Flemingdon Park (44)
270       Flemingdon Park (44)
301       Flemingdon Park (44)
317       Flemingdon Park (44)
367       Flemingdon Park (44)
684       Flemingdon Park (44)
686       Flemingdon Park (44)
696       Flemingdon Park (44)
1437      Flemingdon Park (44)
1825      Flemingdon Park (44)
2405      Flemingdon Park (44)
2687      Flemingdon Park (44)
2693      Flemingdon Park (44)
2694      Flemingdon Park (44)
2930      Flemingdon Park (44)
3001      Flemingdon Park (44)
3172      Flemingdon Park (44)
3174      Flemingdon Park (44)
3393      Flemingdon Park (44)
3402      Flemingdon Park (44)
3548      Flemingdon Park (44)
3789      Flemingdon Park (44)
3852      Flemingdon Park (44)
4013      Flemingdon Park (44)
4202      Flemingdon Park (44)
                  ...         
136472    Flemingdon Park (44)
136550  

In [16]:
# tried with a few different ID's, the names are similar!

In [17]:
# we won't join these dataframes yet because each neighborhood has several tracts (and we can't average tract ids)
# so instead, we'll first get the metrics (eg. average income) for each cansus tracts, so that we can then average the metrics across tracts per neighborhood

#### We want to fetch the census data from this api
#### https://www12.statcan.gc.ca/rest/census-recensement/CPR2016.json?lang=E&dguid={dguid}
#### which provides metrics given a dguid (which represents a geographical area)
#### but first we need to get the dguid for each census tract from this api:

In [18]:
tracts_response = requests.get('https://www12.statcan.gc.ca/rest/census-recensement/CR2016Geo.json?lang=E&geos=CT&cpt=35')
# where CT indicates census tracts, and 35 indicates ontario

In [19]:
# won't convert to json nicely, look at data
tracts_response.text

'//{"COLUMNS":["GEO_UID","PROV_TERR_ID_CODE","PROV_TERR_NAME_NOM","GEO_ID_CODE","GEO_NAME_NOM","GEO_TYPE","GEO_GNR_SF","GEO_GNR_LF","GEO_DQ"],"DATA":[["2016S05075050001.01","35","Ontario","5050001.01","0001.01",null,3.9,5.6,"00001"],["2016S05075050001.04","35","Ontario","5050001.04","0001.04",null,4.6,4.8,"00001"],["2016S05075050001.05","35","Ontario","5050001.05","0001.05",null,3.0,5.7,"00000"],["2016S05075050001.06","35","Ontario","5050001.06","0001.06",null,2.8,5.1,"00000"],["2016S05075050001.07","35","Ontario","5050001.07","0001.07",null,2.0,3.6,"00000"],["2016S05075050001.08","35","Ontario","5050001.08","0001.08",null,2.5,4.0,"00000"],["2016S05075050002.01","35","Ontario","5050002.01","0002.01",null,2.4,6.2,"00000"],["2016S05075050002.02","35","Ontario","5050002.02","0002.02",null,4.8,3.2,"00000"],["2016S05075050002.04","35","Ontario","5050002.04","0002.04",null,1.5,3.4,"00000"],["2016S05075050002.05","35","Ontario","5050002.05","0002.05",null,3.3,4.7,"00000"],["2016S05075050002.0

In [20]:
tracts_text = tracts_response.text.replace('//','')
tracts_text

'{"COLUMNS":["GEO_UID","PROV_TERR_ID_CODE","PROV_TERR_NAME_NOM","GEO_ID_CODE","GEO_NAME_NOM","GEO_TYPE","GEO_GNR_SF","GEO_GNR_LF","GEO_DQ"],"DATA":[["2016S05075050001.01","35","Ontario","5050001.01","0001.01",null,3.9,5.6,"00001"],["2016S05075050001.04","35","Ontario","5050001.04","0001.04",null,4.6,4.8,"00001"],["2016S05075050001.05","35","Ontario","5050001.05","0001.05",null,3.0,5.7,"00000"],["2016S05075050001.06","35","Ontario","5050001.06","0001.06",null,2.8,5.1,"00000"],["2016S05075050001.07","35","Ontario","5050001.07","0001.07",null,2.0,3.6,"00000"],["2016S05075050001.08","35","Ontario","5050001.08","0001.08",null,2.5,4.0,"00000"],["2016S05075050002.01","35","Ontario","5050002.01","0002.01",null,2.4,6.2,"00000"],["2016S05075050002.02","35","Ontario","5050002.02","0002.02",null,4.8,3.2,"00000"],["2016S05075050002.04","35","Ontario","5050002.04","0002.04",null,1.5,3.4,"00000"],["2016S05075050002.05","35","Ontario","5050002.05","0002.05",null,3.3,4.7,"00000"],["2016S05075050002.06"

In [21]:
tracts_data = json.loads(tracts_text)
pprint(tracts_data)

{'COLUMNS': ['GEO_UID',
             'PROV_TERR_ID_CODE',
             'PROV_TERR_NAME_NOM',
             'GEO_ID_CODE',
             'GEO_NAME_NOM',
             'GEO_TYPE',
             'GEO_GNR_SF',
             'GEO_GNR_LF',
             'GEO_DQ'],
 'DATA': [['2016S05075050001.01',
           '35',
           'Ontario',
           '5050001.01',
           '0001.01',
           None,
           3.9,
           5.6,
           '00001'],
          ['2016S05075050001.04',
           '35',
           'Ontario',
           '5050001.04',
           '0001.04',
           None,
           4.6,
           4.8,
           '00001'],
          ['2016S05075050001.05',
           '35',
           'Ontario',
           '5050001.05',
           '0001.05',
           None,
           3.0,
           5.7,
           '00000'],
          ['2016S05075050001.06',
           '35',
           'Ontario',
           '5050001.06',
           '0001.06',
           None,
           2.8,
           5.1,
        

           '00000'],
          ['2016S05075050170.08',
           '35',
           'Ontario',
           '5050170.08',
           '0170.08',
           None,
           1.2,
           1.1,
           '00000'],
          ['2016S05075050170.09',
           '35',
           'Ontario',
           '5050170.09',
           '0170.09',
           None,
           2.0,
           2.6,
           '00000'],
          ['2016S05075050170.10',
           '35',
           'Ontario',
           '5050170.10',
           '0170.10',
           None,
           2.5,
           4.4,
           '00000'],
          ['2016S05075050170.11',
           '35',
           'Ontario',
           '5050170.11',
           '0170.11',
           None,
           1.4,
           1.9,
           '00000'],
          ['2016S05075050170.12',
           '35',
           'Ontario',
           '5050170.12',
           '0170.12',
           None,
           2.0,
           3.0,
           '00000'],
          ['2016S05075050170.

           '5320105.10',
           '0105.10',
           None,
           2.5,
           4.0,
           '00000'],
          ['2016S05075320105.12',
           '35',
           'Ontario',
           '5320105.12',
           '0105.12',
           None,
           1.5,
           3.0,
           '00000'],
          ['2016S05075320105.14',
           '35',
           'Ontario',
           '5320105.14',
           '0105.14',
           None,
           1.8,
           2.4,
           '00000'],
          ['2016S05075320105.15',
           '35',
           'Ontario',
           '5320105.15',
           '0105.15',
           None,
           2.4,
           5.6,
           '00000'],
          ['2016S05075320105.16',
           '35',
           'Ontario',
           '5320105.16',
           '0105.16',
           None,
           2.1,
           4.5,
           '00000'],
          ['2016S05075320105.17',
           '35',
           'Ontario',
           '5320105.17',
           '0105.17',
   

           '35',
           'Ontario',
           '5350095.00',
           '0095.00',
           None,
           4.8,
           7.6,
           '00101'],
          ['2016S05075350096.01',
           '35',
           'Ontario',
           '5350096.01',
           '0096.01',
           None,
           2.8,
           4.2,
           '00100'],
          ['2016S05075350096.02',
           '35',
           'Ontario',
           '5350096.02',
           '0096.02',
           None,
           3.8,
           5.2,
           '00101'],
          ['2016S05075350097.01',
           '35',
           'Ontario',
           '5350097.01',
           '0097.01',
           None,
           5.9,
           6.3,
           '00101'],
          ['2016S05075350097.03',
           '35',
           'Ontario',
           '5350097.03',
           '0097.03',
           None,
           4.9,
           6.7,
           '00101'],
          ['2016S05075350097.04',
           '35',
           'Ontario',
           

          ['2016S05075350220.00',
           '35',
           'Ontario',
           '5350220.00',
           '0220.00',
           None,
           3.4,
           1.8,
           '00000'],
          ['2016S05075350221.01',
           '35',
           'Ontario',
           '5350221.01',
           '0221.01',
           None,
           2.2,
           1.6,
           '00000'],
          ['2016S05075350221.02',
           '35',
           'Ontario',
           '5350221.02',
           '0221.02',
           None,
           5.0,
           3.7,
           '00000'],
          ['2016S05075350222.01',
           '35',
           'Ontario',
           '5350222.01',
           '0222.01',
           None,
           2.8,
           5.1,
           '00000'],
          ['2016S05075350222.02',
           '35',
           'Ontario',
           '5350222.02',
           '0222.02',
           None,
           5.1,
           6.9,
           '00001'],
          ['2016S05075350223.01',
           '35',

...

           '00000'],
          ['2016S05075350804.07',
           '35',
           'Ontario',
           '5350804.07',
           '0804.07',
           None,
           7.8,
           3.5,
           '00100'],
          ['2016S05075350804.08',
           '35',
           'Ontario',
           '5350804.08',
           '0804.08',
           None,
           1.7,
           3.9,
           '00000'],
          ['2016S05075350804.10',
           '35',
           'Ontario',
           '5350804.10',
           '0804.10',
           None,
           3.0,
           4.7,
           '00000'],
          ['2016S05075350804.11',
           '35',
           'Ontario',
           '5350804.11',
           '0804.11',
           None,
           5.0,
           5.9,
           '00000'],
          ['2016S05075350804.12',
           '35',
           'Ontario',
           '5350804.12',
           '0804.12',
           None,
           2.5,
           4.8,
           '00000'],
          ['2016S05075350804.

           '00000'],
          ['2016S05075390008.00',
           '35',
           'Ontario',
           '5390008.00',
           '0008.00',
           None,
           3.0,
           4.5,
           '00000'],
          ['2016S05075390009.00',
           '35',
           'Ontario',
           '5390009.00',
           '0009.00',
           None,
           6.2,
           7.6,
           '00001'],
          ['2016S05075390010.00',
           '35',
           'Ontario',
           '5390010.00',
           '0010.00',
           None,
           3.8,
           6.0,
           '00000'],
          ['2016S05075390011.00',
           '35',
           'Ontario',
           '5390011.00',
           '0011.00',
           None,
           3.6,
           6.0,
           '00000'],
          ['2016S05075390012.01',
           '35',
           'Ontario',
           '5390012.01',
           '0012.01',
           None,
           5.8,
           4.3,
           '00000'],
          ['2016S05075390012.

           '5410152.00',
           '0152.00',
           None,
           2.9,
           3.1,
           '00000'],
          ['2016S05075410153.00',
           '35',
           'Ontario',
           '5410153.00',
           '0153.00',
           None,
           1.7,
           2.3,
           '00000'],
          ['2016S05075430001.00',
           '35',
           'Ontario',
           '5430001.00',
           '0001.00',
           None,
           3.4,
           5.1,
           '00000'],
          ['2016S05075430002.01',
           '35',
           'Ontario',
           '5430002.01',
           '0002.01',
           None,
           6.1,
           7.3,
           '00000'],
          ['2016S05075430002.02',
           '35',
           'Ontario',
           '5430002.02',
           '0002.02',
           None,
           2.0,
           1.6,
           '00000'],
          ['2016S05075430002.03',
           '35',
           'Ontario',
           '5430002.03',
           '0002.03',
   

           5.3,
           7.2,
           '00001'],
          ['2016S05075590027.00',
           '35',
           'Ontario',
           '5590027.00',
           '0027.00',
           None,
           4.3,
           8.1,
           '00100'],
          ['2016S05075590028.00',
           '35',
           'Ontario',
           '5590028.00',
           '0028.00',
           None,
           3.7,
           9.1,
           '00001'],
          ['2016S05075590029.00',
           '35',
           'Ontario',
           '5590029.00',
           '0029.00',
           None,
           6.8,
           9.4,
           '00202'],
          ['2016S05075590030.00',
           '35',
           'Ontario',
           '5590030.00',
           '0030.00',
           None,
           5.4,
           9.3,
           '00101'],
          ['2016S05075590031.00',
           '35',
           'Ontario',
           '5590031.00',
           '0031.00',
           None,
           10.1,
           5.8,
           '01101

           '35',
           'Ontario',
           '5900001.01',
           '0001.01',
           None,
           2.1,
           3.1,
           '00000'],
          ['2016S05075900001.02',
           '35',
           'Ontario',
           '5900001.02',
           '0001.02',
           None,
           2.7,
           3.6,
           '00000'],
          ['2016S05075900002.00',
           '35',
           'Ontario',
           '5900002.00',
           '0002.00',
           None,
           3.2,
           5.0,
           '00000'],
          ['2016S05075900003.00',
           '35',
           'Ontario',
           '5900003.00',
           '0003.00',
           None,
           5.1,
           5.3,
           '00000'],
          ['2016S05075900004.00',
           '35',
           'Ontario',
           '5900004.00',
           '0004.00',
           None,
           2.7,
           5.1,
           '00000'],
          ['2016S05075900005.00',
           '35',
           'Ontario',
           

In [22]:
# to df
tracts_df = pd.DataFrame.from_dict(tracts_data['DATA'])
tracts_df.columns=tracts_data['COLUMNS']

In [23]:
# extract columns we need - only dguid and tract id
tracts_df = tracts_df[['GEO_UID', 'GEO_ID_CODE']]
tracts_df.head()

,GEO_UID,GEO_ID_CODE
0,2016S05075050001.01,5050001.01
1,2016S05075050001.04,5050001.04
2,2016S05075050001.05,5050001.05
3,2016S05075050001.06,5050001.06
4,2016S05075050001.07,5050001.07


In [24]:
len(tracts_df)
# 2376 tracts in ontario, no need to use all

2376

#### Now we have the dguid for each tract code
#### but this is for all of ontario, lets join with the neighborhood conversion df so that we only keep toronto ones

In [25]:
# rename the tract column to be similar to the conversion df so that we can join them
tracts_df = tracts_df.rename(columns={'GEO_ID_CODE':'census_tract'})

In [26]:
# toronto_tracts = conversion_df.join(tracts_df.set_index('census_tract'), on='census_tract')

# error due to joining on different data types

In [27]:
type(tracts_df['census_tract'][0])

str

In [28]:
type(conversion_df['census_tract'][1])

numpy.float64

In [29]:
# one is str and one is float

In [30]:
# cast to str and try joining again
conversion_df['census_tract'] = conversion_df['census_tract'].map("{0:.2f}".format)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [31]:
toronto_tracts = conversion_df.join(tracts_df.set_index('census_tract'), on='census_tract', how='inner')
toronto_tracts.head()

,census_tract,hood_id,neighbourhood,GEO_UID
0,5350001.00,70,South Riverdale,2016S05075350001.00
1,5350002.00,77,Waterfront Communities-The Island,2016S05075350002.00
2,5350003.00,85,South Parkdale,2016S05075350003.00
3,5350004.00,85,South Parkdale,2016S05075350004.00
4,5350005.00,85,South Parkdale,2016S05075350005.00


In [32]:
toronto_tracts = toronto_tracts.rename(columns={'GEO_UID':'dguid'})

In [33]:
toronto_tracts.head()

,census_tract,hood_id,neighbourhood,dguid
0,5350001.00,70,South Riverdale,2016S05075350001.00
1,5350002.00,77,Waterfront Communities-The Island,2016S05075350002.00
2,5350003.00,85,South Parkdale,2016S05075350003.00
3,5350004.00,85,South Parkdale,2016S05075350004.00
4,5350005.00,85,South Parkdale,2016S05075350005.00


In [34]:
len(toronto_tracts)
# 494 dguids (down from 531, some were not in the census tracts data, possibly due to change of tract codes over years)

494

In [37]:
# output to csv
toronto_tracts.to_csv('../output/hoods_to_tracts.csv', index=False)

In [38]:
# conversion is ready to be used by analysis notebooks